
# 教程1：本学期的学习准备工作

## 一、Anaconda的安装和IPython-notebook的安装配置

1. Anaconda是python科学计算的集成。
   - 下载Anaconda，下载地址：http://continuum.io/downloads
   - 安装到/opt/anaconda2目录
   - 编辑文件~/.bashrc，添加一行：
   ```bash
   export PATH=/opt/anaconda2/bin:$PATH
   ```
   - 重新打开terminal，选择用pip安装ipython-notebook
   ```bash
   /opt/anaconda2/bin/pip install ipython[all]==3.2.1
   ```
   - 除此之外，你还需要用pip安装bash_kernel
   ```bash
   pip install bash_kernel
   python -m bash_kernel.install
   ```
2. 切换到用户bio
```
su - bio
```

3. 添加环境变量到文件.bashrc:
```
export PATH=/opt/anaconda2/bin:$PATH
```

4. 配置IPython-notebook：

```bash
# create a new profile
ipython profile create nbserver

# generate password and checksum
ipython -c "from IPython.lib import passwd; passwd()"
# enter your password twice, save it and copy the password hash
## Out[1]: 'sha1:[your hashed password here]'
```

5. 这时候在~bio目录下产生了一个新的目录.ipython，修改其下的文件~bio/.ipython/profile_nbserver/ipython_notebook_config.py：
```
c = get_config()   # 本来就存在的，在后面添加
c.NotebookApp.ip = 'localhost'
c.NotebookApp.open_browser = False
c.NotebookApp.port = 8889
c.NotebookApp.base_url = '/ipython'
c.NotebookApp.password = u'sha1:[your hashed password here]'
c.NotebookApp.allow_origin = '*'
```

6. 在~bio目录下添加路径ipython并启动notebook服务：
```
mkdir ~bio/ipython
cd ~bio/ipython
ipython notebook --profile=nbserver
```

7. 配置Apache也就是http，在配置文件/etc/httpd/conf/httpd.conf中加入

```
# IPython
<Location "/ipython">
    ProxyPass http://localhost:8889/ipython
    ProxyPassReverse http://localhost:8889/ipython
</Location>

<Location "/ipython/api/kernels/">
    ProxyPass http://localhost:8889/ipython/api/kernels/
    ProxyPassReverse http://localhost:8889/ipython/api/kernels/
</Location>
# END IPython
```
8. 重启httpd：
```
sevice httpd restart
```

9. 打开浏览器，输入地址 http://localhost:8889/ipython ，输入密码即可进入

## 二、CentOS下的中文支持

1. 用yum安装中文支持
```
yum install "@Chinese Support"
```
2. 在桌面上修改system->preferences->input method配置中文输入法即可。

## 三、Github的安装配置

### 1. 安装git
```
yum install git git-gui
```

### 2. 在[Github](http://github.com)上注册用户

### 3. 产生私有和公共密钥：
```bash
ssh-keygen -t rsa -C "yourname@name.com"
```

### 4. 将id_rsa.pub文件的内容导入[Github](http://github.com)


## 四、安装和管理不同版本的GCC

你可以在一台机器上安装不同版本的GCC，然后用Environment Module进行管理，实现在不同版本之间的自由切换。

### 1. 安装GCC

GCC的安装很简单，但是编译需要花费很长的时间。
   - （1）下载最新版本的GCC： ftp://mirrors.ustc.edu.cn/gnu/gcc/gcc-5.3.0/
   - （2）编译安装GCC到/opt下：
```
tar xzf gcc-5.3.0.tar.gz
cd gcc-5.3.0
./contrib/download_prerequisites
cd ..
mkdir objdir
cd objdir
$PWD/../gcc-5.3.0/configure --prefix=/opt/gcc-5.3.0
make
make install
```
如果在安装过程中出现了类似“fatal error: gnu/stubs-32.h: No such file or directory”的错误，则需要安装glibc-devel.i586：
```
yum -y install glibc-devel.i686 glibc-devel
```

### 2. 安装Environment Modules软件并管理
```
mkdir -p /usr/local/Modules/src
cd /usr/local/Modules/src
wget http://cbb.sjtu.edu.cn/course/bi201/soft/modules-3.2.10.tar.gz
tar xzvf modules-3.2.10.tar.gz
cd modules-3.2.10
yum install tcl-devel
updatedb
./configure & make & make install
```
现在软件被安装在/usr/local/Modules/3.2.10/目录，其下包含执行命令目录bin和module文件模板目录modulefiles，同时还在/etc/profile.d/生成一个文件modules.sh，这是环境配置文件，需要加载：
```
source /etc/profile.d/modules.sh
```

然后我们就可以使用modules进行软件模块管理了：

   -（1）列出存在的模块：```module avail```
   
   -（2）列出已经加载的模块：```module list```
   
   -（3）使用模块目录：```module use 绝对目录名```
   
   -（4）使用模块：```module load bio/samtools/2.5.1```
   
   -（5）删除模块：```module unload/rm bio/samtools/2.5.1```
   
### 3. 模块文件的写法

这是一个模块文件的模板：
```
#%Module1.0###################################################
##
## modules bio/samtools-0.5.6
##
## modulefiles/bio/samtools-0.5.6. Written by H. Li.
##
proc ModuleHelp { } {
        global version modroot

        puts stderr "bio/samtools-0.5.6 - sequence alignment map tools"
}

module-whatis   "Operates the sequence alignment results file."

# for Tcl scripts use only
set             prefix          /share/bio/samtools
set             version         0.5.6
set             sys                     x86_64

prepend-path    PATH                    $prefix/bin
prepend-path    PATH                    $prefix/scripts
prepend-path    MANPATH                 $prefix/man
prepend-path    LD_LIBARARY_PATH        $prefix/lib
```